# Title - needed

also needed what are we doing here

In [15]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [16]:
%%sql
select * from sqlite_master where type = 'table'

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [17]:
%%sql
select * from facts limit 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [18]:
%%sql
SELECT min(population),
       max(population),
       min(population_growth),
       max(population_growth)
FROM facts;

 * sqlite:///factbook.db
Done.


min(population),max(population),min(population_growth),max(population_growth)
0,7256490011,0.0,4.02


In [19]:
%%sql
SELECT name
FROM facts
WHERE population = 0

 * sqlite:///factbook.db
Done.


name
Antarctica


In [20]:
%%sql
SELECT name
FROM facts
WHERE population = 7256490011

 * sqlite:///factbook.db
Done.


name
World


The database seems to have entry with aggreated value to represent World population.

I am going to remove the world population for average calculation

In [21]:
%%sql
SELECT avg(population),
       avg(area)
FROM facts
WHERE name != 'World'

 * sqlite:///factbook.db
Done.


avg(population),avg(area)
32242666.56846473,555093.546184739


Now that we have average population and the avg area we are going to look for 
countries that have
- higher than average population
- area less than the average area

In [22]:
%%sql
SELECT name,
       population,
       area
FROM facts
WHERE population >
    (SELECT avg(population)
     FROM facts
     WHERE name != 'World')
  AND area <
    (SELECT avg(area)
     FROM facts
     WHERE name != 'World')
  AND name != 'World'

 * sqlite:///factbook.db
Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


- There are 14 countries out of 216 given in database which are having higher than average population and less than average area
---

In [23]:
%%sql
SELECT name,
       round((cast(area_water AS float) / cast(area_land AS float)), 3) AS water_land_ratio
FROM facts
ORDER BY water_land_ratio DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,water_land_ratio
British Indian Ocean Territory,905.667
Virgin Islands,4.52
Puerto Rico,0.555
"Bahamas, The",0.387
Guinea-Bissau,0.285


- if i use ratio > 1 it should tell which countries have more water than land
- we can use order by desc and show all countries, what I am trying is a filter

Using the above query inside a temporary table it is easier to display the country names only.

In [24]:
%%sql
WITH t AS
  (SELECT name,
          round ((cast(area_water AS float) / cast(area_land AS float)), 3) AS water_land_ratio
   FROM facts)
SELECT name
FROM t
WHERE water_land_ratio > 1;

 * sqlite:///factbook.db
Done.


name
British Indian Ocean Territory
Virgin Islands


Now to find which country will increase its population. Logic I am going to be following is as below:

Next Year Population = Current Pop + (Current Pop * birthrate) + (Current Pop * immigration rate) - (Current Pop * death_rate)

In [29]:
%%sql
SELECT name,
       round ((cast(population AS float) * cast(birth_rate AS float)) + 
              (cast(population AS float)* cast(migration_rate AS float)) - 
              (cast(population AS float) * cast(death_rate AS float))) AS projtd_next_year_population
FROM facts
ORDER BY projtd_next_year_population DESC
LIMIT 25;

 * sqlite:///factbook.db
Done.


ltrim(name),projtd_next_year_population
India,15358304816.0
China,7384421095.0
Nigeria,4531788918.0
Pakistan,3509883483.0
Indonesia,2946487188.0
Ethiopia,2914348497.0
Bangladesh,2701634343.0
United States,2635224685.0
Philippines,2045217114.0
"Congo, Democratic Republic of the",1990728411.0


Using the above query inside a temporary table it is easier to display the country names only and displaying only the top 5 countries

In [26]:
%%sql
WITH t AS
  (SELECT name,
          ((cast(population AS float) * cast(birth_rate AS float)) + 
           (cast(population AS float)* cast(migration_rate AS float)) - 
           (cast(population AS float) * cast(death_rate AS float))) AS projtd_next_year_population
   FROM facts
   ORDER BY projtd_next_year_population DESC)
SELECT name
FROM t
LIMIT 5;

 * sqlite:///factbook.db
Done.


name
India
China
Nigeria
Pakistan
Indonesia


- The above coutries are the top 5 countries which will add more population by next year.
---

Countires with higher death rate than birth rate

- check if death_rate > birth_rate
- check if the ratio of death to birth is greater than the average value

In [28]:
%%sql
SELECT name,
       round ((cast(death_rate AS float) / CAST (birth_rate AS float)), 3) AS ratio_death_birth
FROM facts
WHERE ratio_death_birth >
    (SELECT avg((cast(death_rate AS float) / CAST (birth_rate AS float))) AS avg_ratio
     FROM facts)
ORDER BY ratio_death_birth DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,ratio_death_birth
Bulgaria,1.619
Serbia,1.504
Latvia,1.431
Lithuania,1.413
Hungary,1.39
